<a href="https://colab.research.google.com/github/Yuviworks/Week-1-/blob/main/Garbage_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Import required libraries
import tensorflow as tf
import zipfile
import os
import matplotlib.pyplot as plt


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving TrashType_Image_Dataset.zip to TrashType_Image_Dataset.zip


In [ ]:
# ✅ Extracting the zip file
zip_path = "TrashType_Image_Dataset.zip"
extract_path = "/content/data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset unzipped to:", extract_path)


✅ Dataset unzipped to: /content/data


In [ ]:
# ✅ Correct dataset path
dataset_path = "/content/data/Garbage classification"

# ✅ List class names (should show 6)
import os
print("Classes in dataset:", os.listdir(dataset_path))


Classes in dataset: ['paper', 'metal', 'trash', 'cardboard', 'glass', 'plastic']


In [ ]:
# 🔍 List all subdirectories and help find the correct path
for root, dirs, files in os.walk("/content/data"):
    print(root)


/content/data
/content/data/Garbage classification
/content/data/Garbage classification/paper
/content/data/Garbage classification/metal
/content/data/Garbage classification/trash
/content/data/Garbage classification/cardboard
/content/data/Garbage classification/glass
/content/data/Garbage classification/plastic


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# ✅ Parameters
batch_size = 32
img_height = 224
img_width = 224
validation_split = 0.2
seed = 123

# ✅ Create training dataset
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# ✅ Create validation dataset
val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# ✅ Print class names
class_names = train_ds.class_names
print("📂 Classes:", class_names)


Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.
📂 Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B2

# ✅ Load EfficientNetV2B2 without top layers
base_model = EfficientNetV2B2(
    include_top=False,
    weights="imagenet",
    input_shape=(img_height, img_width, 3)
)
base_model.trainable = False  # Freeze base model

# ✅ Build the full model
model = models.Sequential([
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')  # 6 output classes
])

# ✅ Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ Show model summary
model.summary()


35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b2 (Functional)  │ (None, 7, 7, 1408)     │     8,769,374 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,769,374 (33.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 8,769,374 (33.45 MB)

In [13]:
# ✅ Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 280s 4s/step - accuracy: 0.2010 - loss: 1.7778 - val_accuracy: 0.2455 - val_loss: 1.7294
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 208s 3s/step - accuracy: 0.2219 - loss: 1.7426 - val_accuracy: 0.2455 - val_loss: 1.7337
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - accuracy: 0.1882 - loss: 1.7405 - val_accuracy: 0.2455 - val_loss: 1.7198
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 250s 4s/step - accuracy: 0.2187 - loss: 1.7469 - val_accuracy: 0.2455 - val_loss: 1.7162
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 221s 3s/step - accuracy: 0.2288 - loss: 1.7398 - val_accuracy: 0.2455 - val_loss: 1.7140
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.2416 - loss: 1.7103 - val_accuracy: 0.2851 - val_loss: 1.7174
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 210s 3s/step - accuracy: 0.2458 - loss: 1.7232 - val_accuracy: 0.2455 - val_loss: 1.7126
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 210s 3s/step - accuracy: 0.2351 - loss: 1.7304 - val_accuracy: 0.2515 - v